In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
9,application_1606928535611_0010,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hsfs
from graphframes import *

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
node_fg = fs.get_feature_group("node_features", 1)
edge_fg = fs.get_feature_group("edge_features", 1)

In [5]:
edge_df = edge_fg.select(["source","target","is_sar"]).read().toDF("src", "dst", "is_sar")

In [6]:
node_df = node_fg.select(["acct_id","prior_sar"]).read().toDF("id", "prior_sar")

In [25]:
edge_df.where(edge_df.is_sar == 1).show()

+----+----+------+
| src| dst|is_sar|
+----+----+------+
|2298|5688|     1|
|8627|9601|     1|
|2756|8359|     1|
|7605|7702|     1|
|5891|7377|     1|
|6787|1661|     1|
|8485|7950|     1|
|5324|2177|     1|
|4919|4616|     1|
|4170|1589|     1|
|6483|1590|     1|
| 123|2275|     1|
|6173|9851|     1|
|9300|2182|     1|
|2344|5470|     1|
|9149|4651|     1|
| 312|4046|     1|
|5819|2300|     1|
|3792|1964|     1|
|7235|3191|     1|
+----+----+------+
only showing top 20 rows

In [7]:
 # Now lets construct the graph
g = GraphFrame(node_df,edge_df)

In [20]:
sc.setCheckpointDir("hdfs:///Projects/amlsim/Logs/sc")
cc = g.connectedComponents()

In [21]:
cc.show()

+---+---------+---------+
| id|prior_sar|component|
+---+---------+---------+
|  0|        0|        0|
|  1|        0|        1|
|  2|        0|        2|
|  3|        0|        2|
|  4|        0|        2|
|  5|        0|        2|
|  6|        0|        2|
|  7|        0|        2|
|  8|        0|        2|
|  9|        0|        2|
| 10|        0|        2|
| 11|        0|        2|
| 12|        0|        2|
| 13|        0|        2|
| 14|        0|        2|
| 15|        0|        2|
| 16|        0|        2|
| 17|        0|        2|
| 18|        0|        2|
| 19|        0|        2|
+---+---------+---------+
only showing top 20 rows

In [26]:
cc.where(cc.id == 5688).show()

+----+---------+---------+
|  id|prior_sar|component|
+----+---------+---------+
|5688|        1|        2|
+----+---------+---------+

In [28]:
cc.where(cc.component == 2).count()

9998

In [29]:
scc = g.stronglyConnectedComponents(20)

In [30]:
scc.where(scc.id == 5688).show()

+----+---------+---------+
|  id|prior_sar|component|
+----+---------+---------+
|5688|        1|     2298|
+----+---------+---------+

In [32]:
scc.where(scc.component == 2298).show()

+----+---------+---------+
|  id|prior_sar|component|
+----+---------+---------+
|5688|        1|     2298|
|2298|        1|     2298|
+----+---------+---------+